In [ ]:
!git clone https://github.com/fzuphoria/enf1170_gr8.git

In [ ]:
%cd /content/enf1170_gr8


In [ ]:
import pandas as pd

#visualization tools
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

In [ ]:
import os
steam_data = pd.read_csv('steam.csv')
# Define a function to convert owners to exponential notation
def convert_to_exponential(value):
    numbers = value.split("-")
    result = ''
    result =  "{:.0e}".format(float(numbers[0].replace(',', ''))) + '-'+  "{:.0e}".format(float(numbers[1].replace(',', '')))
  
    return result
# get mean value of number of owners
def convert_mean(value):
    numbers = value.split("-")
    result = (int(numbers[0]) + int(numbers[1]))/2
    return int(result)

# Apply the function to the owners column
steam_data['owners_exp'] = steam_data['owners'].apply(convert_to_exponential)
steam_data['owners_mean'] = steam_data['owners'].apply(convert_mean)
steam_data['genres'] = steam_data['genres'].str.lower().apply(lambda x: x.split(';')[0]) # convert lower and split get first
steam_data['categories'] = steam_data['categories'].str.lower() # convert lower
steam_data.head()

In [ ]:
steam_data.describe()


In [ ]:
# Group the games by publisher and calculate the sum of playtime
total_playtime_by_publisher = steam_data.groupby('publisher')['average_playtime'].sum()

# Sort the publishers based on total playtime in descending order
sorted_publishers = total_playtime_by_publisher.sort_values(ascending=False)

# Select the top N publishers to display in the plot (e.g., top 10)
top_publishers = sorted_publishers.head(10)

# Create a bar plot
plt.figure(figsize=(12, 6))
top_publishers.plot(kind='bar')
plt.xlabel('Publisher')
plt.ylabel('Total Playtime')
plt.title('Most Played Games by Publisher')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Group the data by genre and calculate the sum of average playtime
most_played_by_genre = steam_data.groupby('genres')['average_playtime'].mean()

# Filter the genres with average playtime above 100 hours
most_played_by_genre = most_played_by_genre[most_played_by_genre > 50]

# Sort the genres based on total playtime in descending order
most_played_by_genre = most_played_by_genre.sort_values(ascending=False)

# Display the most played games by genre
print(most_played_by_genre)

In [ ]:
# Plot a bar chart
plt.figure(figsize=(10, 6))
most_played_by_genre.plot(kind='bar')

plt.xlabel('Genre')
plt.ylabel('Average Playtime')
plt.title('Distribution of Playtime by Genre')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
most_played_by_genre.plot(kind='pie', autopct='%1.1f%%')

plt.title('Distribution of Playtime by Genre')
plt.ylabel('')
plt.show()

In [ ]:
import numpy as np
# Create a new column to categorize games support
steam_data['controller_support'] = np.where(steam_data['categories'].str.contains('controller support'), True, False)

# Group by game type and calculate the mean playtime
average_playtime_by_controller_support= steam_data.groupby('controller_support')['owners_mean'].mean()
# Plot a pie chart
plt.figure(figsize=(6, 6))
plt.pie(average_playtime_by_controller_support, labels=['No Controller Support', 'Controller Support'], autopct='%1.1f%%')
plt.title('Distribution of Games by Controller Support')

plt.show()

In [ ]:
# 0-20 , 20-50, 50 above parası az olanlara daha çok kişi almıştır.

# Define price ranges
price_ranges = ['0-10', '11-20', '21-30', '31-40', '41-50', '50+']

# Categorize games based on price range
steam_data['price_range'] = pd.cut(steam_data['price'], bins=[0, 10, 20, 30, 40, 50, float('inf')], labels=price_ranges)

# Group by price range and calculate the sum of owners
owners_by_price_range = steam_data.groupby('price_range')['owners_mean'].sum()

# Display the result
print(owners_by_price_range)

In [ ]:

plt.figure(figsize=(10, 12))

# Plot the bar chart
plt.bar(owners_by_price_range.index, owners_by_price_range.values)
plt.xlabel('Price Range')
plt.ylabel('Total Owners')
plt.title('Comparison of Owners based on Price Ranges')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

# Plot the pie chart
plt.pie(owners_by_price_range.values, labels=owners_by_price_range.index, autopct='%1.1f%%')
plt.title('Comparison of Owners based on Price Ranges')
plt.show()


In [ ]:
'''
The release date (year) of a game on Steam has an impact on its popularity, 
as measured by the number of positive user reviews, with more recent games receiving higher ratings compared to older games.
'''
steam_data['release_date'] = pd.to_datetime(steam_data['release_date'])

# Group by release date and calculate the mean positive ratings
average_positive_ratings_by_release_date = steam_data.groupby(pd.Grouper(key='release_date', freq='1Y'))['positive_ratings'].mean()

# Create a bar plot
plt.figure(figsize=(10, 6))
average_positive_ratings_by_release_date.plot(kind='bar')
plt.xlabel('Release Date')
plt.ylabel('Positive Ratings')
plt.title('Positive Ratings for Release Date (1-year intervals)')
plt.xticks(rotation=45)

# Modify x-axis tick labels
tick_labels = [str(label.year) for label in average_positive_ratings_by_release_date.index]
plt.xticks(range(len(tick_labels)), tick_labels)

plt.show()

In [ ]:
'''
The release date (year) of a game on Steam has an impact on its popularity, 
as measured by the number of positive user reviews, with more recent games receiving higher ratings compared to older games.
'''
steam_data['release_date'] = pd.to_datetime(steam_data['release_date'])

# Group by release date and calculate the mean positive ratings
average_positive_ratings_by_release_date = steam_data.groupby(pd.Grouper(key='release_date', freq='1Y'))['negative_ratings'].mean()

# Create a bar plot
plt.figure(figsize=(10, 6))
average_positive_ratings_by_release_date.plot(kind='bar')
plt.xlabel('Release Date')
plt.ylabel('Negative Ratings')
plt.title('Negative Ratings for Release Date (1-year intervals)')
plt.xticks(rotation=45)

# Modify x-axis tick labels
tick_labels = [str(label.year) for label in average_positive_ratings_by_release_date.index]
plt.xticks(range(len(tick_labels)), tick_labels)

plt.show()

In [ ]:
'''
The platform on which a game is available on Steam affects its popularity,
 as measured by the number of positive user reviews. Games available on multiple platforms
 tend to receive more positive ratings compared to games exclusive to a single platform
'''
# Group by number of platforms and calculate the mean positive ratings
average_positive_ratings_by_platforms = steam_data.groupby('platforms')['positive_ratings'].mean()

# Create a bar plot
plt.figure(figsize=(10, 6))
average_positive_ratings_by_platforms.plot(kind='bar')
plt.xlabel('Number of Platforms')
plt.ylabel('Average Positive Ratings')
plt.title('Average Positive Ratings by Number of Platforms')
plt.xticks(rotation=45)

plt.show()


In [ ]:
plt.figure(figsize=(10, 10))

# Plot the pie chart
plt.pie(average_positive_ratings_by_platforms, labels=average_positive_ratings_by_platforms.index, autopct='%1.1f%%')
plt.title('Average Positive Ratings by Number of Platforms')
plt.show()


In [ ]:
'''
"The presence of multiplayer functionality in a game on Steam is associated with
 higher average playtime compared to single-player games."

'''
import numpy as np

# Create a new column to categorize games as multiplayer or single-player
steam_data['game_type'] = np.where(steam_data['steamspy_tags'].str.contains('Multiplayer'), 'Multiplayer', 'Single-player')

# Group by game type and calculate the mean playtime
average_playtime_by_game_type = steam_data.groupby('game_type')['average_playtime'].mean()

# Create a bar plot
plt.figure(figsize=(10, 6))
average_playtime_by_game_type.plot(kind='bar')
plt.xlabel('Game Type')
plt.ylabel('Average Playtime')
plt.title('Average Playtime by Game Type')
plt.xticks(rotation=0)

plt.show()


In [ ]:
plt.figure(figsize=(6, 6))

# Plot the pie chart
plt.pie(average_playtime_by_game_type, labels=average_playtime_by_game_type.index, autopct='%1.1f%%')
plt.title('Average Playtime by Game Type')

plt.show()
